In [1]:
import sys
sys.path.append("/home/pablo/tesis/tsm-pipeline/code")

%load_ext autoreload
%autoreload 2

In [2]:
from scripts.xxx import run_tsm, ExperimentSettings
import docker

docker_client = docker.from_env()
defaults = {
    "bash_config_path": "/home/pablo/tesis/tsm-pipeline/code/scripts/config.sh",
    "o11y_db_dir": "/home/pablo/tesis/tsm-pipeline/code/tsm.db",
}
defaults_evalute = {
    "cache_root":"/home/pablo/dbcache",    
}

El experimento de abajo es una versión de prueba para ver tanto que funciona bien la versión dockerizada y su wrapper en python, como tainted path.

In [6]:
test_experiment = ExperimentSettings(
    name="dell_test_run",
    query_type="path",
    project_list_file="/home/pablo/tesis/tsm-pipeline/experiments/tesis/tainted_path/test_4.txt",
    results_dir="/home/pablo/results/dell_test_run",
    **defaults,
)

run_tsm(docker_client, test_experiment, block=True)

2023-04-15 12:07:10,460 - INFO - creating /home/pablo/results/dell_test_run dir. Omitting error if already existing
2023-04-15 12:07:10,461 - INFO - creating /tmp/log dir. Omitting error if already existing


APIError: 400 Client Error for http+docker://localhost/v1.41/containers/create: Bad Request ("invalid mount config for type "bind": bind source path does not exist: /home/pablo/tesis/tsm-pipeline/code/tsm.db")

In [36]:
!cat /tesis/tmp/results/tainted_path_test/averaged-results.csv

"(parameter 0 (member define (global)))","snk",1.0000000000
"(parameter 0 (member readFile *))","snk",1.0000000000
"(parameter 0 (member readFileSync *))","snk",1.0000000000
"(parameter 0 (member readdir *))","snk",1.0000000000
"(parameter 0 (member require (global)))","snk",1.0000000000
"(parameter 0 (member stat *))","snk",1.0000000000
"(parameter 0 (member statSync *))","snk",1.0000000000

Ahora necesito ver por qué no encuentra resultados para 80 proyectos de `NoSQL`. Yo creo que debería ser capaz de encontrar.

In [13]:
nosql_test = ExperimentSettings(
    name="nosql_test_2",
    query_type="nosql",
    project_list_file="/tesis/repos/tsm-pipeline/experiments/tesis/test_nosql_5.txt",
    results_dir="/tesis/tmp/results/nosql_test_2",
    **defaults
)

run_tsm(docker_client, nosql_test, block=True)

2022-12-30 23:01:34,408 - INFO - creating /tesis/tmp/results/nosql_test_2 dir. Omitting error if already existing
2022-12-30 23:01:34,409 - INFO - creating /tesis/tmp/log dir. Omitting error if already existing
2022-12-30 23:01:34,732 - INFO - running at container 0db1b18f8ebb65ffd5de97a5aa228f509533a116d076b2f2e7232bccf067e278. Use `docker logs 0db1b18f8ebb65ffd5de97a5aa228f509533a116d076b2f2e7232bccf067e278 --tail 10 --follow` to follow progress
2022-12-30 23:02:18,675 - INFO - running combine scores
g: /tesis/tmp/results/nosql_test_2/*/NosqlInjectionWorse-*/reprScores.txt


In [15]:
!cat /tesis/tmp/results/nosql_test_2/averaged-results.csv

"(parameter 0 (member find *))","snk",1.0000000000
"(parameter 0 (member findById *))","snk",1.0000000000
"(parameter 0 (member findByIdAndUpdate *))","snk",1.0000000000
"(parameter 0 (member findOne *))","snk",0.6250000000
"(parameter 0 (member json *))","snk",0.4166666700
"(parameter 0 (member updateOne *))","snk",1.0000000000
"(parameter 1 (member findByIdAndUpdate *))","snk",1.0000000000
"(parameter 1 (member render *))","snk",0.5000000000
"(parameter 1 (member updateOne *))","snk",1.0000000000

Se ve que el problema era que invertía los query_name y query_type, y algo de eso no daba resultado alguno. Corriendo denuevo el entrenamiento de 80 proyectos.

## Evaluando el estado de NoSQL

Entrenando con el top 100 de los repos de NoSQL de ATM. Tomo como corte de entrenamiento los primeros 70, y luego los siguientes 30 para evaluar.

In [17]:
nosql_atm_top_100 = ExperimentSettings(
    name="nosql_atm_top_100",
    query_type="nosql",
    project_list_file="/tesis/repos/tsm-pipeline/experiments/tesis/atm_nosql_top100_train.txt",
    results_dir="/tesis/tmp/results/nosql_atm_top_100",
    **defaults
)

run_tsm(docker_client, nosql_atm_top_100, block=True)

2022-12-30 23:08:21,244 - INFO - creating /tesis/tmp/results/nosql_atm_top_100 dir. Omitting error if already existing
2022-12-30 23:08:21,245 - INFO - creating /tesis/tmp/log dir. Omitting error if already existing
2022-12-30 23:08:21,589 - INFO - running at container 3199eeeac34dd16f86cb4f8f63ad80b4241dee29ff22feaae3da3e1f31a87937. Use `docker logs 3199eeeac34dd16f86cb4f8f63ad80b4241dee29ff22feaae3da3e1f31a87937 --tail 10 --follow` to follow progress
2022-12-30 23:28:28,344 - INFO - running combine scores
g: /tesis/tmp/results/nosql_atm_top_100/*/NosqlInjectionWorse-*/reprScores.txt


Del experimento anterior, tengo resultados muy negativos en el sentido que del total de 70 dbs para entrenar, 42 generaron un modelo vacío:
```
Experimento: nosql_atm_top_100
back home
Creado 2022-12-30 23:08:23.361612
Estado: FINISHED at 2022-12-30 23:28:27.929809
Quedan por entrenar: 0
Total: 70
Cantidad de errores: 42
```
A pesar de esto, hubo descubrimientos de reprs para boostear una query. Pasando a la etapa de evaluación de resultados. En total:

In [25]:
!cat /tesis/tmp/results/nosql_atm_top_100/averaged-results.csv|wc -l

57


In [32]:
from scripts.evaluate import evaluate, EvaluationSettings

In [29]:
worse_settings = EvaluationSettings(
    search_path="/tesis/repos/tsm-pipeline/lib-worse/codeql",
    cli_version="2.5.2",
    query_file="/tesis/repos/tsm-pipeline/tsm-atm-pipeline/src/tsm/evaluation/NosqlInjectionWorseTSM.ql",
    external_predicate_file="/tesis/tmp/results/nosql_atm_top_100/averaged-results.csv",
)

evaluate(
    settings=worse_settings,
    output_dir="/tesis/tmp/results/nosql_atm_top_100/worse",
    dbs_path="/tesis/repos/tsm-pipeline/experiments/tesis/atm_nosql_top100_test.txt",
    **defaults_evalute,
)

2022-12-31 00:36:32,635 - INFO - creating /tesis/tmp/results/nosql_atm_top_100/worse dir. Omitting error if already existing
2022-12-31 00:36:33,234 - INFO - Starting processing with 4 processes
2022-12-31 00:36:33,264 - INFO - Evaluating aheckmann - mquery
2022-12-31 00:36:33,263 - INFO - Evaluating 3dbeb41841bfbfcc24d55143816cf7f1 - atl-wdi-10
2022-12-31 00:36:33,263 - INFO - Evaluating amejiarosario - meanshop
2022-12-31 00:36:33,264 - INFO - Evaluating capacitorset - rebridge
2022-12-31 00:38:17,892 - INFO - Evaluating createdd - votingapp
2022-12-31 00:38:22,648 - INFO - Evaluating cleverbeagle - pup
2022-12-31 00:38:25,747 - INFO - Evaluating cablej - race-condition-demo
2022-12-31 00:38:57,626 - INFO - Evaluating bhdouglass - uappexplorer
2022-12-31 00:38:59,571 - INFO - Evaluating bertrand31 - monitaure
2022-12-31 00:39:09,728 - INFO - Evaluating catsmiaow - tested
2022-12-31 00:39:34,717 - INFO - Evaluating cyantarek - django-microservices
2022-12-31 00:39:34,968 - INFO - Eval

In [33]:
v0_settings = EvaluationSettings(
    search_path="/tesis/v0/javascript/ql/lib",
    cli_version="2.10.5",
    query_file="/tesis/repos/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/NoSqlEvaluation.ql",
)

evaluate(
    settings=v0_settings,
    output_dir="/tesis/tmp/results/nosql_atm_top_100/v0",
    dbs_path="/tesis/repos/tsm-pipeline/experiments/tesis/atm_nosql_top100_test.txt",
    **defaults_evalute,
)

2022-12-31 00:44:07,623 - INFO - creating /tesis/tmp/results/nosql_atm_top_100/v0 dir. Omitting error if already existing
2022-12-31 00:44:08,219 - INFO - Starting processing with 4 processes
2022-12-31 00:44:08,251 - INFO - Evaluating aheckmann - mquery
2022-12-31 00:44:08,251 - INFO - Evaluating capacitorset - rebridge
2022-12-31 00:44:08,251 - INFO - Evaluating 3dbeb41841bfbfcc24d55143816cf7f1 - atl-wdi-10
2022-12-31 00:44:08,251 - INFO - Evaluating amejiarosario - meanshop
2022-12-31 00:46:26,488 - INFO - Evaluating createdd - votingapp
2022-12-31 00:46:29,450 - INFO - Evaluating cablej - race-condition-demo
2022-12-31 00:46:30,407 - INFO - Evaluating cleverbeagle - pup
2022-12-31 00:46:57,322 - INFO - Evaluating bhdouglass - uappexplorer
2022-12-31 00:47:00,610 - INFO - Evaluating bertrand31 - monitaure
2022-12-31 00:47:04,697 - INFO - Evaluating catsmiaow - tested
2022-12-31 00:47:14,852 - INFO - Evaluating biaochenxuying - blog-node
2022-12-31 00:47:29,507 - INFO - Evaluating cy

In [34]:
from scripts.calculate_scores import calculate_scores

calculate_scores(
    results_folder="/tesis/tmp/results/nosql_atm_top_100/",
)

2022-12-31 00:50:57,732 - INFO - Result sets sizes: Worse 260, Boosted 115, V0 108, All 357
2022-12-31 00:50:57,738 - INFO - Score results: Precision: 0.0000. Recall: 0.0000. Accuracy: 0.3754


(0.0, 0.0, 0.3753501400560224)

Los scores excepto por accuracy son pesimos. Necesito debuggear si es un problema de cómo calculo los scores, cómo evalúo, o si posta la técnica está dando resultados muy chotos. Quizas también puede que sean las dbs.